<a href="https://colab.research.google.com/github/llohar/Deep-learning/blob/main/Intro_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def load_dataset():
    train_dataset = h5py.File('/content/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('/content/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def predict(X, parameters):
    
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}
    
    x = tf.placeholder("float", [12288, 1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
        
    return prediction

def forward_propagation_for_predict(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters
    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3'] 
                                                           # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                      # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                     # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                     # Z3 = np.dot(W3,Z2) + b3
    
    return Z3

In [2]:
import numpy as np 
import pandas as pd
import math
import h5py
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops


In [3]:
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
# matplotlib inline
print(np.random.seed())

None


In [5]:
# computing loss using tf
y_hat = tf.constant(36,name='y_hat')
y = tf.constant(39,name='y')

loss =tf.Variable((y_hat-y)**2,name='loss')

# when init is run later
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as session: # create a seesion and run the output
  session.run(init)           # initializes the variables
  print(session.run(loss))    # print the loss


9


In [6]:
a = tf.constant(2)
b = tf.constant(10)
c = tf.multiply(a,b)
print(c)

Tensor("Mul:0", shape=(), dtype=int32)


In [7]:
session = tf.Session()
print(session.run(c))

20


In [8]:
# placeholder: change the value of x in the feed_dict
x = tf.placeholder(tf.int64,name='x')
print(session.run(2*x,feed_dict={x:3}))
session.close


6


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x7f45d29ec5d0>>

In [9]:
# Linear Function
def linear_function():
  X = tf.constant(np.random.randn(3,1),name='X')
  W = tf.constant(np.random.randn(4,3),name='W')
  b = tf.constant(np.random.randn(4,1),name='b')
  Y = tf.add(tf.matmul(W,X),b)

  session = tf.Session()
  result = session.run(Y)
  session.close

  return result


In [10]:
print("result = "+str(linear_function()))

result = [[ 4.0388149 ]
 [-2.09658175]
 [ 1.10890612]
 [-1.5458503 ]]


In [11]:
# sigmoid function
def sigmoid(z):
  x = tf.placeholder(tf.float32,name='x')
  sigmoid = tf.sigmoid(x)

  with tf.Session() as session:
    result = session.run(sigmoid,feed_dict ={x:z})
  return result


In [12]:
print(sigmoid(2))

0.8807971


In [13]:
## cost function
def cost(logits,labels):
  z = tf.placeholder(tf.float32,name='z')
  y = tf.placeholder(tf.float32,name='y')
  cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z,labels=y)
  sess = tf.Session()
  cost = sess.run(cost,feed_dict={z:logits,y:labels})
  sess.close()
  return cost

In [14]:
print(cost(5,4))

-14.993284


In [15]:
# one-hot encoding
def one_hot_matrix(labels,C):
  C = tf.constant(C,name='C')
  one_hot_matrix = tf.one_hot(labels,C,axis=0)
  sess = tf.Session()
  one_hot = sess.run(one_hot_matrix)
  sess.close()
  return one_hot



In [16]:
print(one_hot_matrix(3,5))

[0. 0. 0. 1. 0.]


In [20]:
# ones
def ones(shape):
  ones = tf.ones(shape)
  sess = tf.Session()
  ones = sess.run(ones)
  sess.close()
  return ones
print(ones((5,4)))

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
